# 라이브러리

In [28]:
# 데이터 불러오기
import psycopg2
import json

# 데이터 전처리
import pandas as pd
import numpy as np
import re
import datetime

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

# 기타
import warnings
warnings.filterwarnings("ignore")

# 데이터 불러오기

In [33]:
#Cluster된 데이터 불러오기
import pickle
clustered = pd.read_pickle("./data/clustered_apartment.pkl")
clustered.head(5)

,apartment_id,cluster
0,1,10
1,3,5
2,4,10
3,5,3
4,6,10


In [30]:
def call_df(table_name):
    with open('config.json', 'r') as f:
        config = json.load(f)
        
    conn = psycopg2.connect(user = config['USER'],
                              password = config['PASSWORD'],
                              host = config['HOST'],
                              port = config['PORT'],
                              database = config['DATABASE'])
    
    sql = f'SELECT * FROM {table_name}'
    df = pd.read_sql_query(sql, conn)
    conn.close()
    return df

In [34]:
price = call_df('crawling_db.price_table')
df = price.merge(clustered, how='left', on='apartment_id')

In [36]:
df.head(5)

,price_id,apartment_id,area,period,year,month,amount,amount_original,cluster
0,1,8928,160㎡,2020.02,2020,2,2060000000,"20억 6,000",7
1,2,8928,160㎡,2020.01,2020,1,2030000000,"20억 3,000",7
2,3,8928,160㎡,2019.11,2019,11,1950000000,"19억 5,000",7
3,4,8928,160㎡,2019.10,2019,10,1939999999,"19억 4,000",7
4,5,8928,160㎡,2019.07,2019,7,1739999999,"17억 4,000",7


In [37]:
# area => 숫자가 아닌 무언가가 처음 나오는 것 기준으로 앞의 내용
df['area'] = df['area'].apply(lambda x: int(re.split('\D',x)[0]))
df['period'] = pd.to_datetime(df['period'], format='%Y.%m')